In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
!pip install transformers
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install texttable

In [ ]:
!unzip /content/drive/MyDrive/graph_CS.zip -d /content

In [ ]:
#add /content in the end of next 6 lines
!cp /content/drive/MyDrive/GPT_GNN/__init__.py
!cp /content/drive/MyDrive/GPT_GNN/conv.py
!cp /content/drive/MyDrive/GPT_GNN/model.py
!cp /content/drive/MyDrive/GPT_GNN/utils.py
!cp /content/drive/MyDrive/GPT_GNN/data.py
!cp -av /content/drive/MyDrive/GPT_GNN
import sys
from data import *
from model import *
from warnings import filterwarnings

In [ ]:
attr_ratio = 0.5
attr_type = 'text'
neg_samp_num = 255
queue_size = 256
w2v_dir = '/content/drive/MyDrive/w2v_all'
data_dir = '/content/graph_CS.pk'
pretrain_model_dir = '/content/drive/MyDrive/pretrain_model/model'
cuda = 0
sample_depth = 6
sample_width = 128
conv_name = 'hgt'
n_hid = 400
n_heads = 8
n_layers = 3
prev_norm = True
last_norm = True
dropout = 0.2
max_lr = 1e-3
scheduler = 'cycle'
n_epoch = 2
n_pool = 1
n_batch = 32
batch_size = 256
clip = 0.5

In [ ]:
if cuda != -1:
    device = torch.device("cuda:" + str(cuda))
else:
    device = torch.device("cpu")

print('Start Loading Graph Data...')
graph = renamed_load(open(data_dir, 'rb'))
print('Finish Loading Graph Data!')

pre_range   = {t: True for t in graph.times if t != None and t < 2014}
train_range = {t: True for t in graph.times if t != None and t >= 2014  and t <= 2016}
valid_range = {t: True for t in graph.times if t != None and t > 2016  and t <= 2017}
test_range  = {t: True for t in graph.times if t != None and t > 2017}

pre_target_nodes   = []
train_target_nodes = []
target_type = 'paper'
rel_stop_list = ['self', 'rev_PF_in_L0', 'rev_PF_in_L5', 'rev_PV_Repository', 'rev_PV_Patent']

for p_id, _time in graph.node_feature[target_type]['time'].iteritems():
    if _time in pre_range:
        pre_target_nodes += [[p_id, _time]]
    elif _time in train_range:
        train_target_nodes += [[p_id, _time]]
pre_target_nodes = np.array(pre_target_nodes)
train_target_nodes = np.array(train_target_nodes)

In [ ]:
def GPT_sample(seed, target_nodes, time_range, batch_size, feature_extractor):
    np.random.seed(seed)
    samp_target_nodes = target_nodes[np.random.choice(len(target_nodes), batch_size)]
    threshold   = 0.5
    feature, times, edge_list, _, attr = sample_subgraph(graph, time_range, \
                inp = {target_type: samp_target_nodes}, feature_extractor = feature_extractor, \
                    sampled_depth = sample_depth, sampled_number = sample_width)
    rem_edge_list = defaultdict(  #source_type
                        lambda: defaultdict(  #relation_type
                            lambda: [] # [target_id, source_id] 
                                ))
    
    ori_list = {}
    for source_type in edge_list[target_type]:
        ori_list[source_type] = {}
        for relation_type in edge_list[target_type][source_type]:
            ori_list[source_type][relation_type] = np.array(edge_list[target_type][source_type][relation_type])
            el = []
            for target_ser, source_ser in edge_list[target_type][source_type][relation_type]:
                if relation_type not in rel_stop_list and target_ser < batch_size and np.random.random() > threshold:
                    rem_edge_list[source_type][relation_type] += [[target_ser, source_ser]]
                    continue
                el += [[target_ser, source_ser]]
            el = np.array(el)
            edge_list[target_type][source_type][relation_type] = el
            
            if relation_type == 'self':
                continue
            else:
                if 'rev_' in relation_type:
                    rev_relation = relation_type[4:]
                else:
                    rev_relation = 'rev_' + relation_type
                edge_list[source_type]['paper'][rev_relation] = list(np.stack((el[:,1], el[:,0])).T)
                
    '''
        Adding feature nodes:
    '''
    n_target_nodes = len(feature[target_type])
    feature[target_type] = np.concatenate((feature[target_type], np.zeros([batch_size, feature[target_type].shape[1]])))
    times[target_type]   = np.concatenate((times[target_type], times[target_type][:batch_size]))

    for source_type in edge_list[target_type]:
        for relation_type in edge_list[target_type][source_type]:
            el = []
            for target_ser, source_ser in edge_list[target_type][source_type][relation_type]:
                if target_ser < batch_size:
                    if relation_type == 'self':
                        el += [[target_ser + n_target_nodes, target_ser + n_target_nodes]]
                    else:
                        el += [[target_ser + n_target_nodes, source_ser]]
            if len(el) > 0:
                edge_list[target_type][source_type][relation_type] = \
                    np.concatenate((edge_list[target_type][source_type][relation_type], el))


    rem_edge_lists = {}
    for source_type in rem_edge_list:
        rem_edge_lists[source_type] = {}
        for relation_type in rem_edge_list[source_type]:
            rem_edge_lists[source_type][relation_type] = np.array(rem_edge_list[source_type][relation_type])
    del rem_edge_list
          
    return to_torch(feature, times, edge_list, graph), rem_edge_lists, ori_list, \
            attr[:batch_size], (n_target_nodes, n_target_nodes + batch_size)


In [ ]:
def prepare_data(pool):
    jobs = []
    for _ in np.arange(n_batch - 1):
        jobs.append(pool.apply_async(GPT_sample, args=(randint(), pre_target_nodes, pre_range, batch_size, feature_OAG)))
    jobs.append(pool.apply_async(GPT_sample, args=(randint(), train_target_nodes, train_range, batch_size, feature_OAG)))
    return jobs

In [ ]:
pool = mp.Pool(n_pool)
st = time.time()
jobs = prepare_data(pool)
repeat_num = int(len(pre_target_nodes) / batch_size // n_batch)


data, rem_edge_list, ori_edge_list, _, _ = GPT_sample(randint(), pre_target_nodes, pre_range, batch_size, feature_OAG)
node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data
types = graph.get_types()


gnn = GNN(conv_name = conv_name, in_dim = len(graph.node_feature[target_type]['emb'].values[0]) + 401, n_hid = n_hid, \
          n_heads = n_heads, n_layers = n_layers, dropout = dropout, num_types = len(types), \
          num_relations = len(graph.get_meta_graph()) + 1, prev_norm = prev_norm, last_norm = last_norm)

In [ ]:
if attr_type == 'text':  
    from gensim.models import Word2Vec
    w2v_model = Word2Vec.load(w2v_dir)
    n_tokens = len(w2v_model.wv.vocab)
    attr_decoder = RNNModel(n_word = n_tokens, ninp = gnn.n_hid, \
               nhid = w2v_model.vector_size, nlayers = 2)
    attr_decoder.from_w2v(torch.FloatTensor(w2v_model.wv.vectors))
else:
    attr_decoder = Matcher(gnn.n_hid, gnn.in_dim)
    
gpt_gnn = GPT_GNN(gnn = gnn, rem_edge_list = rem_edge_list, attr_decoder = attr_decoder, \
                  neg_queue_size = 0, types = types, neg_samp_num = neg_samp_num, device = device)
gpt_gnn.init_emb.data = node_feature[node_type == node_dict[target_type][1]].mean(dim=0).detach()
gpt_gnn = gpt_gnn.to(device)

best_val   = 100000
train_step = 0
stats = []
optimizer = torch.optim.AdamW(gpt_gnn.parameters(), weight_decay = 1e-2, eps=1e-06, lr = max_lr)

if scheduler == 'cycle':
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, pct_start=0.02, anneal_strategy='linear', final_div_factor=100,\
                        max_lr = max_lr, total_steps = repeat_num * n_batch * n_epoch + 1)
elif scheduler == 'cosine':
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, repeat_num * n_batch, eta_min=1e-6)

In [ ]:
print('Start Pretraining...')
for epoch in np.arange(n_epoch) + 1:
    gpt_gnn.neg_queue_size = queue_size * epoch // n_epoch
    for batch in np.arange(repeat_num) + 1:
        train_data = [job.get() for job in jobs[:-1]]
        valid_data = jobs[-1].get()
        pool.close()
        pool.join()
        pool = mp.Pool(n_pool)
        jobs = prepare_data(pool)
        et = time.time()
        print('Data Preparation: %.1fs' % (et - st))

        train_link_losses = []
        train_attr_losses = []
        gpt_gnn.train()
        for data, rem_edge_list, ori_edge_list, attr, (start_idx, end_idx) in train_data:
            node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data
            node_feature = node_feature.detach()
            node_feature[start_idx : end_idx] = gpt_gnn.init_emb
            node_emb = gpt_gnn.gnn(node_feature.to(device), node_type.to(device), edge_time.to(device), \
                                   edge_index.to(device), edge_type.to(device))

            loss_link, _ = gpt_gnn.link_loss(node_emb, rem_edge_list, ori_edge_list, node_dict, target_type, use_queue = True, update_queue=True)
            if attr_type == 'text':
                loss_attr = gpt_gnn.text_loss(node_emb[start_idx : end_idx], attr, w2v_model, device)
            else:
                loss_attr = gpt_gnn.feat_loss(node_emb[start_idx : end_idx], torch.FloatTensor(attr).to(device))


            loss = loss_link + loss_attr * attr_ratio


            optimizer.zero_grad() 
            loss.backward()
            torch.nn.utils.clip_grad_norm_(gpt_gnn.parameters(), clip)
            optimizer.step()

            train_link_losses += [loss_link.item()]
            train_attr_losses += [loss_attr.item()]
            scheduler.step()
        '''
            Valid
        '''
        gpt_gnn.eval()
        with torch.no_grad():
            data, rem_edge_list, ori_edge_list, attr, (start_idx, end_idx) = valid_data
            node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data
            node_feature = node_feature.detach()
            node_feature[start_idx : end_idx] = gpt_gnn.init_emb
            node_emb = gpt_gnn.gnn(node_feature.to(device), node_type.to(device), edge_time.to(device), \
                                       edge_index.to(device), edge_type.to(device))
            loss_link, ress = gpt_gnn.link_loss(node_emb, rem_edge_list, ori_edge_list, node_dict, target_type, use_queue = False, update_queue=True)
            loss_link = loss_link.item()
            if attr_type == 'text':   
                loss_attr = gpt_gnn.text_loss(node_emb[start_idx : end_idx], attr, w2v_model, device)
            else:
                loss_attr = gpt_gnn.feat_loss(node_emb[start_idx : end_idx], torch.FloatTensor(attr).to(device))

            ndcgs = []
            for i in ress:
                ai = np.zeros(len(i[0]))
                ai[0] = 1
                ndcgs += [ndcg_at_k(ai[j.cpu().numpy()], len(j)) for j in i.argsort(descending = True)]     
                
            valid_loss = loss_link + loss_attr * attr_ratio
            st = time.time()
            print(("Epoch: %d, (%d / %d) %.1fs  LR: %.5f Train Loss: (%.3f, %.3f)  Valid Loss: (%.3f, %.3f)  NDCG: %.3f  Norm: %.3f  queue: %d") % \
                  (epoch, batch, repeat_num, (st-et), optimizer.param_groups[0]['lr'], np.average(train_link_losses), np.average(train_attr_losses), \
                   loss_link, loss_attr, np.average(ndcgs), node_emb.norm(dim=1).mean(), gpt_gnn.neg_queue_size))  
            
        if valid_loss < best_val:
            best_val = valid_loss
            print('UPDATE!!!')
            torch.save(gpt_gnn.state_dict(), pretrain_model_dir)
        stats += [[np.average(train_link_losses),  loss_link, loss_attr, valid_loss]]
